In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("datas/bank-full.csv")

#clean dataframe
df['default'] = df['default'].map({'yes':1 ,'no':0}) 
df['loan'] = df['loan'].map({'yes':1 ,'no':0}) 
df['Target'] = df['Target'].map({'yes':1 ,'no':0}) 
df['housing'] = df['housing'].map({'yes':1 ,'no':0})
df['marital'] = df['marital'].map({'single':0 ,'married':1,'divorced':2})
df['education'] = df['education'].map({'unknown':0 ,'primary':1,'secondary':2,'tertiary':3})
df['contact'] = df['contact'].map({'unknown':0 ,'cellular':1,'telephone':1})
df['poutcome'] = df['poutcome'].map({'unknown':0, 'success':1,'failure':2,'other':3})
df['month'] = df['month'].map({'jan':1, 'feb':2, 'mar':3, 'apr':4, 'may':5, 'jun':6,'jul':7,'aug':8,'sep':9, 'oct':10,'nov':11,'dec':12})


df.drop(["job"], axis=1, inplace=True)

df

,age,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,Target
0,58,1,3,0,2143,1,0,0,5,5,261,1,-1,0,0,0
1,44,0,2,0,29,1,0,0,5,5,151,1,-1,0,0,0
2,33,1,2,0,2,1,1,0,5,5,76,1,-1,0,0,0
3,47,1,0,0,1506,1,0,0,5,5,92,1,-1,0,0,0
4,33,0,0,0,1,0,0,0,5,5,198,1,-1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,1,3,0,825,0,0,1,17,11,977,3,-1,0,0,1
45207,71,2,1,0,1729,0,0,1,17,11,456,2,-1,0,0,1
45208,72,1,2,0,5715,0,0,1,17,11,1127,5,184,3,1,1
45209,57,1,2,0,668,0,0,1,17,11,508,4,-1,0,0,0


In [3]:
df.dtypes
# df.isnull().sum()

age          int64
marital      int64
education    int64
default      int64
balance      int64
housing      int64
loan         int64
contact      int64
day          int64
month        int64
duration     int64
campaign     int64
pdays        int64
previous     int64
poutcome     int64
Target       int64
dtype: object

In [4]:
df['month'].value_counts(dropna=False)


5     13766
7      6895
8      6247
6      5341
11     3970
4      2932
2      2649
1      1403
10      738
9       579
3       477
12      214
Name: month, dtype: int64

In [5]:
# Importation du générateur de rapport de profile
from pandas_profiling import ProfileReport

In [6]:
# profile = ProfileReport(df)
# profile.to_file('bank-full-report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
#split data for the first model

dfClientData = df[['age','marital','education','default','balance','housing','loan','Target']]
dfClientData

,age,marital,education,default,balance,housing,loan,Target
0,58,1,3,0,2143,1,0,0
1,44,0,2,0,29,1,0,0
2,33,1,2,0,2,1,1,0
3,47,1,0,0,1506,1,0,0
4,33,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...
45206,51,1,3,0,825,0,0,1
45207,71,2,1,0,1729,0,0,1
45208,72,1,2,0,5715,0,0,1
45209,57,1,2,0,668,0,0,0


In [12]:
#split data for the second model

dfBankCampaignData = df[['contact','day','month','duration','campaign','pdays','previous','poutcome','Target']]
dfBankCampaignData

,contact,day,month,duration,campaign,pdays,previous,poutcome,Target
0,0,5,5,261,1,-1,0,0,0
1,0,5,5,151,1,-1,0,0,0
2,0,5,5,76,1,-1,0,0,0
3,0,5,5,92,1,-1,0,0,0
4,0,5,5,198,1,-1,0,0,0
...,...,...,...,...,...,...,...,...,...
45206,1,17,11,977,3,-1,0,0,1
45207,1,17,11,456,2,-1,0,0,1
45208,1,17,11,1127,5,184,3,1,1
45209,1,17,11,508,4,-1,0,0,0


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# First model

In [17]:
y = dfClientData.Target
x = dfClientData.drop(['Target'], axis = 1)

x_train,x_test, y_train, y_test = train_test_split(x,y,test_size = .2, random_state = 42)

In [18]:
rf = RandomForestClassifier(n_estimators = 100, random_state = 23, max_features = 2)

In [19]:
rf.fit(X=x_train, y=y_train)

RandomForestClassifier(max_features=2, random_state=23)

In [30]:
#Chaque colonne elle prend en compte à 8% Pclass, 25% le Sex etc, ainsi c'est l'importance des données en entrer 
pd.DataFrame(rf.feature_importances_*100,x_train.columns, columns=['Importance en %'])



,Importance en %
contact,1.641800
day,17.363307
month,11.955046
duration,47.466588
campaign,5.536654
pdays,7.168343
previous,3.094211
poutcome,5.774052


In [31]:
# import shap

# shpv = shap.TreeExplainer(rf).shap_values(x_train)
# shap.summary_plot(shpv, x_train)

In [32]:
def print_score(learn_obj, x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test):
	print("Score train : ", learn_obj.score(x_train, y_train,)*100, "%")
	print("Score test : ", learn_obj.score(x_test, y_test,)*100, "%")
print_score(rf)

Score train :  99.84793187347933 %
Score test :  90.02543403737697 %


# Second model

In [33]:
y = dfBankCampaignData.Target
x = dfBankCampaignData.drop(['Target'], axis = 1)

x_train,x_test, y_train, y_test = train_test_split(x,y,test_size = .2, random_state = 42)

In [34]:
rf = RandomForestClassifier(n_estimators = 100, random_state = 23, max_features = 2)

In [35]:
rf.fit(X=x_train, y=y_train)

RandomForestClassifier(max_features=2, random_state=23)

In [36]:
#Chaque colonne elle prend en compte à 8% Pclass, 25% le Sex etc, ainsi c'est l'importance des données en entrer 
pd.DataFrame(rf.feature_importances_*100,x_train.columns, columns=['Importance en %'])

,Importance en %
contact,1.641800
day,17.363307
month,11.955046
duration,47.466588
campaign,5.536654
pdays,7.168343
previous,3.094211
poutcome,5.774052


In [37]:
print_score(rf)

Score train :  99.84793187347933 %
Score test :  90.02543403737697 %
